# <font color='red'> Data collection - Part 1</font>

In [88]:
import os
import json
import praw

Enter your user name, client id, and client secret to create a read-only reddit instance.

In [90]:
creds = json.load(open("creds.json"))

username = creds.get("username")
clientid = creds.get("clientid")
clientsecret = creds.get("clientsecret")

reddit = praw.Reddit(client_id=clientid,
                     client_secret=clientsecret,
                     user_agent='data collection (by {})'.format(username),
                    redirect_uri='http://localhost:8000',)

## Retrieving and saving data - praw

We can do a search for key words in a specific subreddit or all of them.<br>

For example let's serach for the phrase *I feel guilty* in all subreddits:

In [56]:
import pandas as pd
from datetime import datetime


counter = 0
posts_table = []
column_names = ["Index", "subreddit", "post_time",
                "user_name", "title", "text",
                "num_comments", "score", "ups", "upvote_ratio", 
                "downs", 'downvote', 
                "URL",  
               "over18"]

subreddit = "AutisticAdults"
results = reddit.subreddit(subreddit).new(limit=200)
for p in results: # p is a reddit post
    counter += 1
    # Add the post to the table
    posts_table.append([counter, p.subreddit, datetime.utcfromtimestamp(p.created),
                        p.author, p.title, p.selftext,
                        p.num_comments, p.score, p.ups, p.upvote_ratio,
                        p.downs, p.downvote,
                        p.permalink
                       , p.over_18])
    

posts_df = pd.DataFrame(posts_table, columns=column_names)

In [57]:
posts_df.to_csv("postsASD.csv", index=False) # index=False - do not save the default index column

In [58]:
posts_df.head()

,Index,subreddit,post_time,user_name,title,text,num_comments,score,ups,upvote_ratio,downs,downvote,URL,over18
0,1,AutisticAdults,2023-05-27 10:11:19,stephenharoldsmith,Diagnosed this day last week,I'm just putting this out there. I was diagnos...,0,2,2,1.0,0,<bound method VotableMixin.downvote of Submiss...,/r/AutisticAdults/comments/13t3lnl/diagnosed_t...,False
1,2,AutisticAdults,2023-05-27 09:22:56,ezm_ob,My friend is convenienced that dietary choices...,\nMy friend (doesn't know im autistic) goes to...,0,2,2,1.0,0,<bound method VotableMixin.downvote of Submiss...,/r/AutisticAdults/comments/13t2ta8/my_friend_i...,False
2,3,AutisticAdults,2023-05-27 08:57:34,Solid-Ad-75,Is it ableism when neurotypicals do this?,Sometimes someone will say something that does...,0,3,3,1.0,0,<bound method VotableMixin.downvote of Submiss...,/r/AutisticAdults/comments/13t2e19/is_it_ablei...,False
3,4,AutisticAdults,2023-05-27 06:14:39,luna_bea_tuna,Realisation - why I like cartoons / kids shows...,A sudden realisation while watching cartoons w...,3,7,7,0.9,0,<bound method VotableMixin.downvote of Submiss...,/r/AutisticAdults/comments/13szrun/realisation...,False
4,5,AutisticAdults,2023-05-27 04:33:12,Putrid-Occasion1881,Autism and BPD? *Vent,I got diagnosed with autism as an adult but I ...,1,1,1,1.0,0,<bound method VotableMixin.downvote of Submiss...,/r/AutisticAdults/comments/13sxz75/autism_and_...,False


In [64]:
len(posts_df.user_name.unique())

175

In [70]:
user_asd_original = posts_df.user_name.unique()

## Save all user submissions

In [60]:
users = posts_df['user_name'].unique()
users = [u for u in users if u is not None]
submission_comments_table = []
column_names = ["Index", 'is_post',  "subreddit", "post_time",
                "user_name", "title", "text",
                "num_comments", "score", "ups", "upvote_ratio", 
                "downs", 'downvote', 'likes', 
                "URL",  "over18"]
counter = 0

for user in users:
    submissions = user.submissions.new(limit=100)
    for p in submissions:
        if p.selftext in ["", "[removed]"]:
            continue
        counter += 1
        submission_comments_table.append([counter, "yes", p.subreddit, datetime.utcfromtimestamp(p.created), 
                            p.author, p.title, p.selftext, 
                            p.num_comments, p.score, p.ups, p.upvote_ratio, 
                            p.downs, p.downvote,  p.likes,
                            p.permalink,  p.over_18])


ValueError: 16 columns passed, passed data had 14 columns

In [61]:
posts_df_all = pd.DataFrame(submission_comments_table, columns=column_names)

In [62]:
posts_df_all.to_csv("postsASD_all.csv", index=False) # index=False - do not save the default index column

In [71]:
user_asd_new = posts_df_all.user_name.unique()

In [72]:
## making sure that we only save users who are in asd group
## this should be empty - indeed
[ u for u in user_asd_new if u not in user_asd_original]

[]

#### Manual Filter subreddits which are:
1. related to autism
2. have less than 4 users

In [63]:
subreddits_dict = {}
subreddits = posts_df_all.subreddit.unique()
for sub in subreddits:
    users = posts_df_all[posts_df_all.subreddit == sub].user_name.unique()
    subreddits_dict[sub] = users

sorted_dict = dict(sorted(subreddits_dict.items(), key=lambda x: len(x[1]), reverse=True))

In [65]:
for k, v in sorted_dict.items():
    if len(v) < 3:
        continue
    print(f"subreddit = {k}, #users = {len(v)}")

subreddit = AutisticAdults, #users = 168
subreddit = autism, #users = 32
subreddit = AutismInWomen, #users = 16
subreddit = ADHD, #users = 13
subreddit = AutismTranslated, #users = 13
subreddit = CPTSD, #users = 12
subreddit = aspergers, #users = 12
subreddit = depression, #users = 9
subreddit = raisedbynarcissists, #users = 9
subreddit = AutisticWithADHD, #users = 9
subreddit = aspergirls, #users = 8
subreddit = Anxiety, #users = 7
subreddit = neurodiversity, #users = 7
subreddit = AskDocs, #users = 7
subreddit = AskVet, #users = 6
subreddit = lgbt, #users = 6
subreddit = mentalhealth, #users = 6
subreddit = antiwork, #users = 6
subreddit = NonBinary, #users = 5
subreddit = cats, #users = 5
subreddit = offmychest, #users = 5
subreddit = AuDHDWomen, #users = 5
subreddit = adhdwomen, #users = 5
subreddit = rant, #users = 5
subreddit = NoStupidQuestions, #users = 5
subreddit = TwoXChromosomes, #users = 5
subreddit = ChronicIllness, #users = 4
subreddit = HomeImprovement, #users = 4
subre

In [66]:
filtered_top_subreddits = ["depression", "Anxiety", "raisedbynarcissists", "AskDocs", "AskVet", "antiwork", "cats", "offmychest", "rant", "NoStupidQuestions", "HomeImprovement", "personalfinance", "TrueOffMyChest", "AmItheAsshole", "relationship_advice", "Dreams", "Advice", "dogs"]

### Collect neurotypical posts from top subreddits

In [68]:
import pandas as pd
from datetime import datetime

users_asd = posts_df_all.user_name.unique()
counter = 0
posts_table = []
column_names = ["Index", "subreddit", "post_time",
                "user_name", "title", "text",
                "num_comments", "score", "ups", "upvote_ratio",
                "downs", 'downvote',
                "URL",
               "over18"]

for subreddit in filtered_top_subreddits:
    results = reddit.subreddit(subreddit).new(limit=15)
    for p in results: # p is a reddit post
        if p.selftext in ["", "[removed]", "[deleted]"]:
            continue
        if p.author in ["[deleted]"]:
            continue
        if p.author in users_asd:
            continue
        counter += 1
        # Add the post to the table
        posts_table.append([counter, p.subreddit, datetime.utcfromtimestamp(p.created),
                            p.author, p.title, p.selftext,
                            p.num_comments, p.score, p.ups, p.upvote_ratio,
                            p.downs, p.downvote,
                            p.permalink
                           , p.over_18])

posts_df_nr = pd.DataFrame(posts_table, columns=column_names)

In [73]:
posts_df_nr.to_csv("postsNRTypical.csv", index=False) # index=False - do not save the default index column

In [69]:
posts_df_nr.head()

,Index,subreddit,post_time,user_name,title,text,num_comments,score,ups,upvote_ratio,downs,downvote,URL,over18
0,1,depression,2023-05-27 10:58:24,LostCookie29,Thinking of cutting everyone off,I (25F) recently made a huge sacrifice for my ...,0,1,1,1.0,0,<bound method VotableMixin.downvote of Submiss...,/r/depression/comments/13t4dgg/thinking_of_cut...,False
1,2,depression,2023-05-27 10:51:16,VanaVisera,(F27) I Feel Like A Complete Failure,I’m a 27 year old goth woman who feels like al...,0,2,2,1.0,0,<bound method VotableMixin.downvote of Submiss...,/r/depression/comments/13t497j/f27_i_feel_like...,True
2,3,depression,2023-05-27 10:31:30,Original_Letter_2477,I wish to be a child again,"It was beautiful. Living day by day, going out...",0,2,2,1.0,0,<bound method VotableMixin.downvote of Submiss...,/r/depression/comments/13t3xnl/i_wish_to_be_a_...,False
3,4,depression,2023-05-27 10:01:18,Competitive-Sea-9878,Hi guys im new to this,Umm so i just had a break through in my head b...,0,1,1,1.0,0,<bound method VotableMixin.downvote of Submiss...,/r/depression/comments/13t3fic/hi_guys_im_new_...,False
4,5,depression,2023-05-27 09:59:08,Last_Temperature_55,I quick my antidepressants cold turkey and now...,I quit taking my antidepressants just a couple...,0,1,1,1.0,0,<bound method VotableMixin.downvote of Submiss...,/r/depression/comments/13t3e2x/i_quick_my_anti...,False


## Matching by subreddit - save results

In [84]:
import os
df_all = []
for subreddit in filtered_top_subreddits:
    sub_df_nr = posts_df_nr[posts_df_nr.subreddit == subreddit]
    sub_df_nr["ASD"] = False
    sub_df_asd = posts_df_all[posts_df_all.subreddit == subreddit]
    sub_df_asd["ASD"] = True
    df_both = pd.concat([sub_df_asd, sub_df_nr], axis=0)
    df_both.drop(columns=["Index", "downvote", "is_post"], inplace=True)
    df_both.reset_index(inplace=True)
    df_both.to_csv(os.path.join("posts",f"{subreddit}_both.csv"), index=False)
    df_all.append(df_both)

C:\Users\ilani\AppData\Local\Temp\ipykernel_15808\2597613869.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df_nr["ASD"] = False
C:\Users\ilani\AppData\Local\Temp\ipykernel_15808\2597613869.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df_asd["ASD"] = True
C:\Users\ilani\AppData\Local\Temp\ipykernel_15808\2597613869.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [85]:
df_all = pd.concat(df_all, axis=0)
df_all.reset_index(inplace=True)
df_all.to_csv("posts_ASD_Typical_filtered.csv", index=False)

In [87]:
df_all.head(20)

,level_0,index,subreddit,post_time,user_name,title,text,num_comments,score,ups,upvote_ratio,downs,likes,URL,over18,ASD
0,0,30,depression,2023-02-20 17:34:29,luna_bea_tuna,I make things worse,I'm at the point where I can see my family wou...,0,1,1,1.00,0,None,/r/depression/comments/117f0hz/i_make_things_w...,True,True
1,1,34,depression,2022-12-16 08:46:15,luna_bea_tuna,How do you stay patient waiting for medication...,Hello hello. I am 4-6 weeks into brintellix fo...,0,1,1,1.00,0,None,/r/depression/comments/zna3os/how_do_you_stay_...,False,True
2,2,35,depression,2022-11-30 07:25:44,luna_bea_tuna,I can’t do this anymore,I have a 3 year old. And everyday is crying an...,4,2,2,0.67,0,None,/r/depression/comments/z8kw6y/i_cant_do_this_a...,False,True
3,3,37,depression,2022-11-15 09:16:53,luna_bea_tuna,Visualized tools for depression,Hi - I suffer from anxiety and depression and ...,0,2,2,1.00,0,None,/r/depression/comments/yvrt9b/visualized_tools...,False,True
4,4,38,depression,2022-11-12 17:23:20,luna_bea_tuna,What is my depression telling me?,Is it correct to think that when I feel depres...,4,2,2,0.67,0,None,/r/depression/comments/ytd5zd/what_is_my_depre...,False,True
5,5,40,depression,2022-08-17 08:05:30,luna_bea_tuna,No way out.,"My partner and I have no interests in common, ...",2,2,2,1.00,0,None,/r/depression/comments/wqj5is/no_way_out/,False,True
6,6,550,depression,2023-01-03 08:22:19,GhostAmethyst,Is it Winter blues or just my life?,I’m gonna keep a lot of details to a minimum j...,0,1,1,1.00,0,None,/r/depression/comments/1022fut/is_it_winter_bl...,False,True
7,7,561,depression,2022-05-14 12:31:39,GhostAmethyst,Consequences of Hygiene Struggles,Been feeling incredibly insecure for quite a f...,8,14,14,0.95,0,None,/r/depression/comments/upga0k/consequences_of_...,False,True
8,8,925,depression,2020-12-04 23:39:44,cyborgmonkey-,A hopeless loser,Younger and less qualified people are getting ...,0,2,2,1.00,0,None,/r/depression/comments/k6wf3i/a_hopeless_loser/,False,True
9,9,933,depression,2020-09-24 01:28:46,cyborgmonkey-,Overwhelmed.,"I have depression, anxiety, bipolar. A couple ...",0,1,1,1.00,0,None,/r/depression/comments/iynh7u/overwhelmed/,False,True
